In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import librosa
import numpy as np
import os
from glob import glob
import json
from tqdm import tqdm

In [3]:
audio_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/audio/mono_pickup_mix"
json_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation"
cqt_out_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt"

In [4]:
def harmonic_cqt_stack(y, sr=22050, n_octaves=4, bins_per_semitone=3):
    base_freqs = [
        82.41 * 0.5,  # Sub-harmonic
        82.41 * 1.0,  # E2
        82.41 * 2.0,  # 2nd harmonic
        82.41 * 3.0,
        82.41 * 4.0,
        82.41 * 5.0   # 5th harmonic
    ]

    cqt_stack = []
    for fmin in base_freqs:
        cqt = librosa.cqt(
            y, sr=sr,
            hop_length=512,
            fmin=fmin,
            n_bins=n_octaves * 12 * bins_per_semitone,
            bins_per_octave=12 * bins_per_semitone
        )
        cqt_db = librosa.amplitude_to_db(np.abs(cqt))
        cqt_stack.append(cqt_db)

    stacked_cqt = np.stack(cqt_stack, axis=0)  # shape: [6, freq_bins, T]
    return stacked_cqt

In [10]:
audio_path = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/audio/mono_pickup_mix/00_BN1-129-Eb_comp_mix.wav"

y_test, sr_test = librosa.load(audio_path, sr=22050)

cqt = harmonic_cqt_stack(y_test, sr_test)
print("CQT shape:", cqt.shape)

CQT shape: (6, 144, 962)


In [11]:
os.makedirs(cqt_out_dir, exist_ok=True)

audio_files = sorted([f for f in os.listdir(audio_dir) if f.endswith(".wav")])

for file in tqdm(audio_files):
    file_path = os.path.join(audio_dir, file)
    y, sr = librosa.load(file_path, sr=22050)

    # Harmonic CQT çıkar
    harm_cqt = harmonic_cqt_stack(y, sr=sr)  # [6, F, T]

    # Kaydet
    filename = os.path.splitext(file)[0]
    out_path = os.path.join(cqt_out_dir, f"{filename}_cqt.npy")
    np.save(out_path, harm_cqt),

100%|██████████| 360/360 [04:54<00:00,  1.22it/s]


In [20]:
audio_files = [os.path.splitext(f)[0] for f in os.listdir(audio_dir) if f.endswith(".wav")]
cqt_files = [os.path.splitext(f)[0].replace("_cqt", "") for f in os.listdir(cqt_out_dir) if f.endswith(".npy")]

# Eksik olanları bul
missing = sorted(list(set(audio_files) - set(cqt_files)))

print("Eksik CQT dosyaları:")
for f in missing:
    print(f)

Eksik CQT dosyaları:


In [21]:
import numpy as np

cqt_path = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_BN1-129-Eb_comp_mix_cqt.npy"

cqt = np.load(cqt_path)
print("CQT shape:", cqt.shape)
print("Min:", np.min(cqt), "Max:", np.max(cqt), "Mean:", np.mean(cqt))

CQT shape: (6, 144, 962)
Min: -72.978874 Max: 14.927688 Mean: -45.44909


In [25]:
import numpy as np, os

lengths = []

for fname in os.listdir(cqt_out_dir):
    if not fname.endswith(".npy"):
        continue
    path = os.path.join(cqt_out_dir, fname)
    cqt = np.load(path)
    lengths.append(cqt.shape[1])  # frekans ekseni (F)

print(set(lengths))  # farklı F boyutlarını yaz


{144}


Eğitimden önce veriyi normalize et

In [24]:
import pandas as pd

mismatches = []
differences = []

for fname in sorted(os.listdir(cqt_out_dir)):
    if not fname.endswith(".npy"):
        continue

    cqt_path = os.path.join(cqt_out_dir, fname)
    base_name = fname.replace("_cqt.npy", "")
    json_name = f"{base_name}_fretnet.json"
    json_path = os.path.join(json_dir, json_name)

    if not os.path.exists(json_path):
        mismatches.append((fname, "MISSING_JSON", "-"))
        continue

    try:
        cqt = np.load(cqt_path)
        with open(json_path, "r") as f:
            data = json.load(f)

        times = data.get("times", [])
        cqt_len = cqt.shape[1]
        times_len = len(times)
        diff = cqt_len - times_len

        if diff != 0:
            mismatches.append((fname, cqt_len, times_len))
            differences.append(diff)

    except Exception as e:
        mismatches.append((fname, "ERROR", str(e)))

df = pd.DataFrame(mismatches, columns=["Filename", "CQT_frames", "Times_frames"])

print("📊 Uyuşmayan dosya sayısı:", len(df))
if differences:
    print("🧮 En küçük fark:", min(differences))
    print("🧮 En büyük fark:", max(differences))
else:
    print("🧮 Farklılık yok.")

print(df.head())

📊 Uyuşmayan dosya sayısı: 360
🧮 En küçük fark: -1998
🧮 En büyük fark: -531
                         Filename  CQT_frames  Times_frames
0  00_BN1-129-Eb_comp_mix_cqt.npy         144          1044
1  00_BN1-129-Eb_solo_mix_cqt.npy         144          1044
2  00_BN1-147-Gb_comp_mix_cqt.npy         144           918
3  00_BN1-147-Gb_solo_mix_cqt.npy         144           918
4   00_BN2-131-B_comp_mix_cqt.npy         144          1368


Eğitimden önce veriyi kırp